In [1]:
import seaborn as sns
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
%pylab inline
import os
from __future__ import print_function
from __future__ import division
import os, sys
sys.path.append(os.path.abspath('.'))
import preprocessing_time_series as pts


os.chdir(pts.working_dir())

Populating the interactive namespace from numpy and matplotlib


In [2]:
os.listdir('.')

['.DS_Store',
 'cliente_tabla.csv',
 'cliente_tabla.csv.zip',
 'producto_tabla.csv',
 'producto_tabla.csv.zip',
 'producto_tabla_w_clusters.csv',
 'sample_submission.csv',
 'sample_submission.csv.zip',
 'States',
 'test.csv',
 'test.csv.zip',
 'test_preprocessed.csv',
 'town_state.csv',
 'town_state.csv.zip',
 'train.csv',
 'train.csv.zip',
 'train_JALISCO_timeseries.csv',
 'train_sample.csv',
 'train_SONORA_timeseries.csv']

# Как соотносятся категории

In [3]:
df = pd.read_csv('train_sample.csv')
df.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,brand,weight,pieces,Has_choco,Has_vanilla,Has_multigrano,NombreCliente,Log_Demanda,Dev_proxima_by_uni,No_remains
0,3,1110,7,3301,15766,1238,4,39.32,0,0.0,...,BIM,140.0,2.0,0,0,0,PUESTO DE PERIODICOS LAZARO,1.609438,NaN,0
1,3,1110,7,3301,319641,1125,50,480.00,0,0.0,...,TR,255.0,10.0,0,0,0,GUADALAJARA,3.931826,NaN,0
2,3,1110,7,3301,319641,2233,23,458.62,0,0.0,...,BIM,640.0,NaN,0,0,0,GUADALAJARA,3.178054,NaN,0
3,3,1110,7,3301,819816,2233,14,279.16,0,0.0,...,BIM,640.0,NaN,0,0,0,SANATORIO ESPANOL SA,2.708050,NaN,0
4,3,1110,7,3301,1163700,31506,6,37.50,0,0.0,...,MLA,113.0,6.0,0,0,0,PUESTO DEGALLETAS,1.945910,NaN,0


In [4]:
df.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Dev_uni_proxima',
       u'Dev_proxima', u'Demanda_uni_equil', u'Town', u'State',
       u'NombreProducto', u'brand', u'weight', u'pieces', u'Has_choco',
       u'Has_vanilla', u'Has_multigrano', u'NombreCliente', u'Log_Demanda',
       u'Dev_proxima_by_uni', u'No_remains'],
      dtype='object')

In [13]:
from itertools import combinations
import codecs

categories = [u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Town', u'State',
       u'NombreProducto', u'brand', u'NombreCliente']

def check(cat_pair):
    uniques = df.groupby(cat_pair[0])[cat_pair[1]].nunique()
    return uniques.max()==1

with codecs.open('categories_hierarhy.tsv', 'w', 'utf-8') as f:
    for pair_id in combinations(range(len(categories)), 2):
        pair = [categories[i] for i in pair_id]
        less = check(pair)
        greater = check(pair[::-1])
        if less & greater:
            symb = '='
        elif less:
            symb = '<'
        elif greater:
            symb = '>'
        else:
            symb = '?'
        if symb != '?':
            print (pair[0] + '\t' + symb + '\t' + pair[1])
        f.write(pair[0] + '\t' + symb + '\t' + pair[1] + '\n')


Agencia_ID	<	Town
Agencia_ID	<	State
Producto_ID	=	NombreProducto
Producto_ID	<	brand
Town	<	State
NombreProducto	<	brand


# Какие новые категории есть в тестовых данных

In [15]:
test = pd.read_csv('test_preprocessed.csv',index_col=0)
print(test.shape)
test.head()

(7055802, 16)


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town,State,NombreProducto,brand,weight,pieces,Has_choco,Has_vanilla,Has_multigrano,NombreCliente
id,,,,,,,,,,,,,,,,
0,11,4037,1,2209,4639078,35305,2499 REYNOSA MARINELA,TAMAULIPAS,Barritas Fresa 75g Prom MTB MLA 35305,MTB MLA,75.0,NaN,0,0,0,NaN
1,11,2237,1,1226,4705135,1238,2402 TUXTLA,CHIAPAS,Panquecito Gota Choc 2p 140g BIM 1238,BIM,140.0,2.0,0,0,0,NaN
2,10,2045,1,2831,4549769,32940,2163 LA PIEDAD,MICHOACAN,Principe Choc Bco 10p 110g Prom MTB MLA 32940,MTB MLA,110.0,10.0,0,0,0,NaN
3,11,1227,1,4448,4717855,43066,2066 TOLUCA PONIENTE,ESTADO DE MEXICO,Pay Pina 1p 85g MTB MLA 43066,MTB MLA,85.0,1.0,0,0,0,NaN
4,11,1219,1,1130,966351,1277,2042 AG. TEPOZOTLAN,ESTADO DE MEXICO,Nito Edicion Especial 62g BIM 1277,BIM,62.0,NaN,0,0,0,LOS PINITOS
